# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [4]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату.

In [136]:
def foldl2(f, x0, lst):
    return foldr(lambda x, y: f(y, x), x0, lst[::-1])

In [160]:
def foldr2(f, x0, lst):
    return foldl(lambda x, y: f(y, x), x0,  lst[::-1])

Тесты:

In [169]:
from random import randint
import sys

for i in range(10000):
    a = [randint(1, 10) for i in range(10)]

    divide_function = lambda x, y: x / y
    sum_function = lambda x, y: x + y
    concatenate = lambda x, y: f'{x}{y}'

    # Check foldl2 function
    assert foldl2(divide_function, 1, a) - foldl(divide_function, 1, a) < sys.float_info.epsilon
    assert foldl2(sum_function, 0, a) - foldl(sum_function, 0, a) < sys.float_info.epsilon
    assert foldl2(concatenate, '', a) == foldl(concatenate, '', a)
    
    # Check foldr2 function
    assert foldr2(divide_function, 1, a) - foldr(divide_function, 1, a) < sys.float_info.epsilon
    assert foldr2(sum_function, 0, a) - foldr(sum_function, 0, a) < sys.float_info.epsilon
    assert foldr2(concatenate, '', a) == foldr(concatenate, '', a)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

## Наивная реализация

Генерируем все перестановки строки b и проверяем, входит ли хотя бы одна из них в строку a.

Пусть, n - длина строки a, m - длина строки b, тогда сложность алгоритма $O(n * m!)))$

In [183]:
from itertools import permutations
from functools import reduce
import operator

def check_inv(a, b):
    # Get all permutations and make strings from tuples
    perm = list(map(lambda x: reduce(operator.add, x), permutations(b)))
    for p in perm:
        if p in a:
            return True
    return False

In [188]:
a = 'abcrotm'
b = 'tro'
check_inv(a, b)

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [75]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    @property
    def is_leaf(self):
        return self.left is None and self.right is None
        
    def __list__(self):
        left_list = []
        right_list = []
        
        if self.left:
            left_list = list(self.left)
        if self.right:
            right_list = list(self.right)
        if self.is_leaf:
            return [self.value]
        return left_list + right_list
        
    def __iter__(self):
        return iter(self.__list__())
    
    def __str__(self):
        if self.is_leaf:
            return str(self.value)
        left_str = self.left.__str__() or ''
        right_str = self.right.__str__() or ''
        return left_str + ' ' + right_str
    
    def __repr__(self):
        return 'Leafs are: ' + ' '.join(list(map(str, self.__list__())))
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

print(list(tree) == [3, 4, 2])
print(tree)
print(repr(tree))

True
3 4 2
Leafs are: 3 4 2


# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

Также добавил:

- вещественные числа
- функции одного аргумента (квадратный корень, синус, косинус)

In [74]:
from math import sqrt, sin, cos

OPERATORS = {'+': (1, lambda x, y: x + y), '-': (1, lambda x, y: x - y),
             '*': (2, lambda x, y: x * y), '/': (2, lambda x, y: x / y),
            'sin': (3, lambda x: sin(x)), 'cos': (3, lambda x: cos(x)),
             'sqrt': (3, lambda x: sqrt(x))}

FUNCTIONS = {'sin': lambda x: sin(x), 'cos': lambda x: cos(x),
             'sqrt': lambda x: sqrt(x)}

def get_tokens(formula_string):
    number = ''
    func = ''
    for s in formula_string:
        if s in 'sincossqrt':
            func += s
        elif func:
            yield func
            func = ''
        if s.isdigit() or s == '.':
            number += s  
        elif number: 
            yield float(number) 
            number = ''
        if s in OPERATORS or s in "()":
            yield s 
        
    if number:
        yield float(number) 


def shunting_yard(parsed_formula):
    stack = []
    for token in parsed_formula:
        if token in FUNCTIONS:
            stack.append(token)
        elif token in OPERATORS: 
            while stack and stack[-1] != "(" and OPERATORS[token][0] <= OPERATORS[stack[-1]][0]:
                yield stack.pop()
            stack.append(token)
        elif token == ")":
            while stack:
                x = stack.pop()
                if x == "(":
                    break
                yield x
        elif token == "(":
            stack.append(token)
        else:
            yield token
    while stack:
        yield stack.pop()
        
def calculate(rpn):
    # rpn - reverse polish notation
    stack = []
    for token in rpn:
        if isinstance(token, float):
            stack.append(token)
        elif token in FUNCTIONS:
            x = stack.pop()
            stack.append(FUNCTIONS[token](x))
        elif token in OPERATORS:
            y = stack.pop()
            x = stack.pop()
            try:
                stack.append(OPERATORS[token][1](x, y))
            except ZeroDivisionError:
                raise ZeroDivisionError('Деление на 0')
    return stack[0]

def calc(expr):
    return calculate(shunting_yard(get_tokens(expr)))

# default tests
ex1 = '2 *(15 - 3 * 4) - 2'
ex2 = '(5 *10 - 30 + 2) /2 - 1'
ex3 = '(10 + (20 + (30 + (1*0)) ) )'

# sin/cos/sqrt tests
ex4 = 'sqrt(1 + sqrt(7 + 1 * 2) * 5)'
ex5 = 'sin(sin(sin(cos(sqrt(2)))))'

# float tests
ex6 = '1.2 + 5.23478329 - sqrt(2.432890043289)'

assert calc(ex1) == eval(ex1)
assert calc(ex2) == eval(ex2)
assert calc(ex3) == eval(ex3)
assert calc(ex4) == eval(ex4)
assert calc(ex5) == eval(ex5)
assert calc(ex6) == eval(ex6)

try:
    calc('1 / 0')
    print('Not ok')
except:
    print('Zero division check is OK')

Zero division check is OK
